In [0]:
from music21 import converter, instrument,note, chord
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import numpy as np

In [0]:
import glob
import numpy

In [0]:
def get_notes():
  notes =[]
  for file in glob.glob("/content/drive/My Drive/input_music/*.mid"):
    midi = converter.parse(file)
    print("Parsing %s" % file)
    notes_to_parse = None
    try:
      s2 = instrument.partitionByInstrument(midi)
      notes_to_parse = s2.parts[0].recurse()
    except:
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element,note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element,chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
  with open('/content/drive/My Drive/data/notes','wb') as filepath:
    pickle.dump(notes,filepath)
  return notes
    

In [0]:
def prepare_sequences(notes,n_vocab):
  sequence_length = 100
  pitchnames = sorted(set(item for item in notes))
  note_to_int = dict((note,number) for number,note in enumerate(pitchnames))

  network_input = []
  network_output = []
  for i in range(0,len(notes)- sequence_length,1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append([note_to_int[sequence_out]])

  n_patterns = len(network_input)
  network_input = numpy.reshape(network_input,(n_patterns,sequence_length,1))
  network_input = network_input / float(n_vocab)

  network_output = np_utils.to_categorical(network_output)
  return (network_input,network_output)
  

In [0]:
def create_network(network_input,n_vocab):
  model = Sequential()
  model.add(LSTM(512,
                 input_shape=(network_input.shape[1],network_input.shape[2]),
                 return_sequences = True))
  model.add(Dropout(0.3))
  model.add(LSTM(512,return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
  return model
  

In [0]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "trained_models/weights-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history = model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
    return history

In [0]:
if __name__ == '__main__':
    
    # Pre processing of data
    notes = get_notes()
    n_vocab = len(set(notes))
    
    # Data formation
    network_input, network_output = prepare_sequences(notes, n_vocab)
    # Model architecture
    model = create_network(network_input, n_vocab)
    # Treaining
    history = train(model, network_input, network_output)
    # Ploting
    plot_history(history)

Parsing /content/drive/My Drive/input_music/10.mid
Parsing /content/drive/My Drive/input_music/11.mid
Parsing /content/drive/My Drive/input_music/13.mid
Parsing /content/drive/My Drive/input_music/12.mid
Parsing /content/drive/My Drive/input_music/1.mid
Parsing /content/drive/My Drive/input_music/28.mid
Parsing /content/drive/My Drive/input_music/25.mid
Parsing /content/drive/My Drive/input_music/26.mid
Parsing /content/drive/My Drive/input_music/27.mid
Parsing /content/drive/My Drive/input_music/24.mid
Parsing /content/drive/My Drive/input_music/22.mid
Parsing /content/drive/My Drive/input_music/2.mid
Parsing /content/drive/My Drive/input_music/20.mid
Parsing /content/drive/My Drive/input_music/23.mid
Parsing /content/drive/My Drive/input_music/21.mid
Parsing /content/drive/My Drive/input_music/19.mid
Parsing /content/drive/My Drive/input_music/14.mid
Parsing /content/drive/My Drive/input_music/15.mid
Parsing /content/drive/My Drive/input_music/16.mid
Parsing /content/drive/My Drive/i

In [0]:
import matplotlib.pyplot as plt


In [0]:
def plot_history(history):
  loss_hist = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
  val_loss_hist = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
  acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
  val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

  if len(loss_list) ==0:
    print('Loss is missing in history')
    return
  epochs = range(1,len(history.history[loss_list[0]])+1)
  plt.figure(1)
  for l in loss_list:
    plt.plot(epochs,history.history[1],'b',label='Training loss ('+str(str(format(history.history[l][-1],'.5f'))+')'))
  for l in val_loss_list:
    plt.plot(epochs,history.history[l],'g',label='Validation loss (' +str(str(format(history.history[l][-1],'.5f'))+')'))

  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  #Accuracy
  plt.figure(2)
  for l in acc_list:
    plt.plot(epochs,history.history[l],'b',label='Training Accuracy (' +str(str(format(history.history[l][-1],'.5f'))+')')

  for l in val_acc_list:
    plt.plot(epochs,history.history[l],'g',label='Validation accuracy ('+ str(str(format(history.history[l][-1],'.5f'))+')')

  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

In [0]:
def prepare_sequences(notes,pitchnames,n_vocab):
  note_to_int = dict((note,number) for number, note in enumerate(pitchnames))
  sequence_length = 100
  network_input = []
  output = []
  for i in range(0,len(notes)- sequence_length,1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

  n_patterns = len(netwokr_input)
  normalized_input = numpy.reshape(network_input,(n_patterns,sequence_length,1))
  normalized_input  normalized_input /float(n_vocab)

  return (network_input,normalized_input)

def create_network(network_input,n_vocab):
  model = Sequential()
  model.add(LSTM(512,
                 input_shape = (network_input.shape[1],network_input.shape[2]),
                 return_sequences = True))
  model.add(Dropout(0.3))
  model.add(LSTM(512,return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
  model.load_weights('weights.hdf5')
  return model

def generate_nodes(model,network_input,pitchnames,n_vocab):
  start = numpy.random.randint(0,len(network_input)-1)
  int_to_note = dict((number,state) for number, note in enumerate(pitchnames))
  pattern = network_input[start]
  prediction_output = []

  for note_index in range(1000):
    prediction_input = numpy.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input,verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
  return prediction_output

def create_midi(prediction_output):
  offset = 0
  output_notes = []

  for pattern in prediction_output:
    if('.' in pattern) or pattern.isdigit():
      notes_in_chord = patter.split('.')
      notes =[]
      for current_note in notes_in_chord:
        new_note = note.Note(int(current_note))
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
      new_chord = chord.Chord(notes)
      new_chord.offset = offset
      output_notes.append(new_chord)
    else:
      new_note = note.Note(pattern)
      new_note.offset = offset
      new_note.storedInstrument  = instrument.Piano()
      output_notes.append(new_note)
    offset +=0.5
  midi_stream  = stream.Stream(output_notes)
  midi_stream.write('midi',fp='generated_music/test_output.mid')


if __name__ =='__main__':
  with open('/content/drive/My Drive/data/notes','rb') as filepath:
    notes = pickle.load(filepath)

  pitchnames = sorted(set(item for item in notes))
  n_vocab = len(set(notes))
  network_input, normalized_input = prepare_sequences(notes,pitchnames,n_vocab)
  model = create_network(normalized_input,n_vocab)

  prediction_output = generate_notes(model,network_input,pitchnames,n_vocab)

  create_midi(prediction_output)
